In [1]:
import sys

In [2]:
#Custom package environment
sys.path.insert(0,'/KIMEL/tigrlab/projects/jjeyachandra/gmsh-sdk/lib/')
sys.path.insert(0,'/home/jjeyachandra/simnibs_2.1.2/miniconda2/envs/simnibs_env/lib/python2.7/site-packages')
sys.path.insert(0,'/home/jjeyachandra/simnibs_2.1.2/Python_modules/src')

In [3]:
import numpy as np
import os

In [4]:
#Add cornell library
sys.path.insert(0, '/projects/jjeyachandra/Cornell-MOE/')

In [61]:
from moe.optimal_learning.python.python_version.domain import TensorProductDomain
from moe.optimal_learning.python.geometry_utils import ClosedInterval
from moe.optimal_learning.python.data_containers import HistoricalData, SamplePoint
from examples import synthetic_functions

In [56]:
#Define an objective function using their example func
objective_func = synthetic_functions.Hartmann3()

In [57]:
#Make an n-dimensional domain containing closed intervals
search_domain = TensorProductDomain([ClosedInterval(objective_func._search_domain[i,0],
                                                   objective_func._search_domain[i,1]) 
                                    for i in np.arange(objective_func._search_domain.shape[0])])

In [58]:
#Array to fill in with initial points
init_pts = np.zeros( (objective_func._num_init_pts, objective_func._dim) )
init_pts[:, :objective_func._dim] = search_domain.generate_uniform_random_points_in_domain(
                                            objective_func._num_init_pts)
print(init_pts)

[[ 0.81723034  0.86540908  0.89887275]
 [ 0.66009505  0.25215713  0.45954503]
 [ 0.09638129  0.36695075  0.29724925]]


In [76]:
#Evaluate at init pts using noise-less sampling
observations = np.array([objective_func.evaluate_true(p).sum() for p in init_pts])
print(observations)

[ 9.81882693  1.64821418  0.85939432]


In [79]:
#Store observations into a data_container and display
init_data = HistoricalData(dim = objective_func._dim, num_derivatives=0)
init_data.append_sample_points([(c,v,0) for c,v in zip(init_pts,observations)])
init_data.json_payload()

{'points_sampled': [{'point': [0.81723033874107887,
    0.86540908342922673,
    0.89887275414073065],
   'value': [9.8188269335235905],
   'value_var': 0.0},
  {'point': [0.66009504779020034, 0.25215712838013293, 0.45954503378893574],
   'value': [1.6482141792435554],
   'value_var': 0.0},
  {'point': [0.09638129153038133, 0.36695075393403798, 0.29724924814858822],
   'value': [0.85939431739889982],
   'value_var': 0.0}]}

In [ ]:
#Prior model initialization
